In [23]:
# General
import numpy as np
import pickle as pk
import math

# For optimization
import pygmo as pg

# For computing acceleration and potential
import polyhedral_gravity as model

# For working with the mesh
import mesh_utility
import tetgen
import meshio as mio
import openmesh as om

# For Plotting
import pyvista as pv
import mesh_plotting
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from matplotlib import cm


class basic_udp:
    
    def fitness(self,x):

        # Initial information
        r = np.transpose(x[0:3]) # Start Position
        v = np.transpose(x[3:6]) # Initial velocity

        # Creating the mesh (TetGen)
        mesh, vertices, faces = mesh_utility.create_mesh()

        # Defining additional parameters
        density = 533           # https://sci.esa.int/web/rosetta/-/14615-comet-67p
        target_altitude = 10    # In unit: [km]
        t_end = 100            # Final time
        t_0 = 0                 # Starting time
        d_t = 1                # Number of steps along orbit 

        # Array containing times for summation
        time_list = np.arange(t_0,t_end,d_t)

        # Fitness value (to be maximized)
        fit_val = 0

        
        for t in time_list:
            print("I'm at time: ", t)
            # Retrieve information at current position
            altitude = compute_Altitude(r, mesh)
            potential, a, tensor = model.evaluate(vertices, faces, density, r)
            a = np.array(a)

            # Computing velocity and position for next time-step
            
            ##################
            print("Value of v: ", v, type(v), ",   Value of a: ", a, type(a), ",    Value of r: ", r, type(r), ",    Value of d_t: ", d_t, type(d_t))
            ##################

            v_n = v + d_t * a
            r_n = r + d_t * v

            # Evaluate current position
            fit_val += abs(target_altitude - altitude)

            # Update current velocity and position
            v = v_n
            r = r_n
            
            print("This is the fit value: ", fit_val)
        return [fit_val]


    def get_bounds(self):
        # Bounds for the initial domain (i.e bounds for possible x-chromosomes)
        #   min=[r_x,r_y,r_z,v_x,v_y,v_z], max=[r_x,r_y,r_z,v_x,v_y,v_z]
        # Good starting boundaries could e.g be right outside the most protruding part of the the mesh 
        return ([4000,4000,4000,10,10,10],[10000,10000,10000,800,800,800])



def compute_Altitude(r,mesh):
    # Obtaining closest distance from mesh to satelite: 
    #    See PyVista function: https://docs.pyvista.org/api/core/_autosummary/pyvista.DataSet.find_closest_point.html
    index = mesh.grid.find_closest_point(r) 
    closest_point = mesh.grid.points[index]
    altitude = np.linalg.norm(closest_point-r)
    return altitude



def get_trajectory(mesh, vertices, faces, x):

    #to be finished.

    # Initial information
    r = np.transpose(x[0:3]) # Start Position
    v = np.transpose(x[3:6]) # Initial velocity

    # Defining additional parameters
    density = 533           # https://sci.esa.int/web/rosetta/-/14615-comet-67p
    target_altitude = 10    # In unit: [km]
    t_end = 100             # Final time
    t_0 = 0                 # Starting time
    d_t = 1                 # Number of steps along orbit 

    # Array containing times for summation
    time_list = np.arange(t_0,t_end,d_t)
    
    # Numpy Arrays to store trajectory information
    r_store = np.zeros()
    v_store = np.zeros()

    # Add starting position to memory
    r_store[:,1] = r
    v_store[:,1] = v
    i = 2

    for t in time_list:

        # Retrieve information at current position
        altitude = compute_Altitude(r, mesh)
        potential, a, tensor = model.evaluate(vertices, faces, density, r)
        a = np.array(a)

        # Computing velocity and position for next time-step
        v_n = v + d_t * a
        r_n = r + d_t * v

        # Update current velocity and position
        v = v_n
        r = r_n

        # Storing updated trajectory information
        r_store[:,i] = r
        v_store[:,i] = v
        i += 1

    return r_store, v_store



def plot_trajectory(mesh, vertices, faces, r_store):

    # Plotting mesh of asteroid/comet
    mesh_plot = pv.Plotter()
    mesh_plot.add_mesh(mesh.grid, show_edges=True)
    mesh_plot.show_bounds(grid='front',location='outer',all_edges=True)

    # Plotting trajectory
    trajectory_plot = pv.PolyData(r_store)
    mesh_plot.add_mesh(trajectory_plot, color=[0.6, 0.2, 0.1])


def plot_evolutions():
    ...




In [24]:

# Defining and executing optimization
prob = pg.problem(basic_udp())
print(prob)

Problem name: <class '__main__.basic_udp'>
	C++ class name: pybind11::object

	Global dimension:			6
	Integer dimension:			0
	Fitness dimension:			1
	Number of objectives:			1
	Equality constraints dimension:		0
	Inequality constraints dimension:	0
	Lower bounds: [4000, 4000, 4000, 10, 10, ... ]
	Upper bounds: [10000, 10000, 10000, 800, 800, ... ]
	Has batch fitness evaluation: false

	Has gradient: false
	User implemented gradient sparsity: false
	Has hessians: false
	User implemented hessians sparsity: false

	Fitness evaluations: 0

	Thread safety: none



In [25]:
algo = pg.algorithm(pg.de(gen = 20))
pop = pg.population(prob = basic_udp(), size = 5)

Reading file...
Physical dimension along x (UN-normalized):  5002.5703125 Km
Tetrahralize...
Writing vtk files...
Writing polyhedron-cpp input files..
Writing FORTRAN files..
Writing other files...
Finished.
I'm at time:  0
Value of v:  [598.79625846 295.87927072 402.91228288] <class 'numpy.ndarray'> ,   Value of a:  [8.94281323e-06 1.04610079e-05 9.94070196e-06] <class 'numpy.ndarray'> ,    Value of r:  [5257.02082174 6423.57200722 5837.15272888] <class 'numpy.ndarray'> ,    Value of d_t:  1 <class 'int'>
This is the fit value:  7116.795605163509
I'm at time:  1
Value of v:  [598.79626741 295.87928118 402.91229282] <class 'numpy.ndarray'> ,   Value of a:  [8.03893790e-06 8.88247475e-06 8.58002487e-06] <class 'numpy.ndarray'> ,    Value of r:  [5855.8170802  6719.45127794 6240.06501175] <class 'numpy.ndarray'> ,    Value of d_t:  1 <class 'int'>
This is the fit value:  14992.460574149967
I'm at time:  2
Value of v:  [598.79627544 295.87929006 402.9123014 ] <class 'numpy.ndarray'> ,   V

In [26]:
pop = algo.evolve(pop)

Reading file...
Physical dimension along x (UN-normalized):  5002.5703125 Km
Tetrahralize...
Writing vtk files...
Writing polyhedron-cpp input files..
Writing FORTRAN files..
Writing other files...
Finished.
I'm at time:  0
Value of v:  [186.61359716 714.95417195 618.53186982] <class 'numpy.ndarray'> ,   Value of a:  [8.63733954e-06 6.16750120e-06 5.48750988e-06] <class 'numpy.ndarray'> ,    Value of r:  [8646.55434516 6354.77937452 5482.64118113] <class 'numpy.ndarray'> ,    Value of d_t:  1 <class 'int'>
This is the fit value:  9275.291412062246
I'm at time:  1
Value of v:  [186.6136058  714.95417812 618.53187531] <class 'numpy.ndarray'> ,   Value of a:  [7.27449763e-06 5.67393202e-06 5.03342560e-06] <class 'numpy.ndarray'> ,    Value of r:  [8833.16794231 7069.73354647 6101.17305095] <class 'numpy.ndarray'> ,    Value of d_t:  1 <class 'int'>
This is the fit value:  19315.41366171086
I'm at time:  2
Value of v:  [186.61361307 714.95418379 618.53188034] <class 'numpy.ndarray'> ,   Va

In [27]:
print(os.getcwd())

/Users/rasmus/Master Thesis/My Repositories/67P-Swarm/Introductory Tests


In [30]:
uda = algo.extract(pg.de)
uda.get_log() 
print(pop.champion_f) 
print(pop.champion_x) 



[1228303.46191298]
[6235.25852267 6102.75563126 5939.98181393   91.39030129   20.83721372
   36.34276697]


In [31]:

x = pop.champion_x

# Creating the mesh and plotting results
mesh, vertices, faces = mesh_utility.create_mesh()

r_store, v_store = get_trajectory(mesh, vertices, faces, x)



Reading file...
Physical dimension along x (UN-normalized):  5002.5703125 Km
Tetrahralize...
Writing vtk files...
Writing polyhedron-cpp input files..
Writing FORTRAN files..
Writing other files...
Finished.


TypeError: get_trajectory() missing 1 required positional argument: 't_end'

In [ ]:
    #altitude = math.dist(closest_point, r)
plot_trajectory(mesh, vertices, faces, r_store)